<header>
   <p  style='font-size:36px;font-family:Arial; color:#F0F0F0; background-color: #00233c; padding-left: 20pt; padding-top: 20pt;padding-bottom: 10pt; padding-right: 20pt;'>
       Energy Consumption Forecasting
  <br>
       <img id="teradata-logo" src="https://storage.googleapis.com/clearscape_analytics_demo_data/DEMO_Logo/teradata.svg" alt="Teradata" style="width: 125px; height: auto; margin-top: 20pt;">
    </p>
</header>

<p style = 'font-size:20px;font-family:Arial;color:#00233C'><b>Introduction</b></p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Electricity consumption is a key driver to a successful energy trading company. But this success is only translated into profits if they can correctly predict how much energy will be consumed. In fact, proper forecasting of market energy demand prevents losses, in case of overselling energy to market, and lost profits, in case of underestimating demand. Not only will an energy company’s only cash flow be affected by false predictions, but the regulator of the energy market can apply fees or even disqualify a trading company for specific periods for frequent inaccurate forecasts. Fortunately, Teradata Vantage and ClearScape Analytics provide the ideal platform to create these predictions. With Vantage’s advanced in-database analytics, time series functions, and AI/ML capabilities, companies can increase their confidence in the predictions.</p>

<p style = 'font-size:18px;font-family:Arial;color:#00233C'><b>Business Values</b></p>

<ul style = 'font-size:16px;font-family:Arial;color:#00233C'>
    <li>Decrease profit losses by over predicting or under predicting the amount of energy consumed.</li>
    <li>Decrease regulations placed on trading companies from the energy market.</li>
    <li>Increase energy consumption prediction accuracy.</li>
</ul>

<p style = 'font-size:18px;font-family:Arial;color:#00233C'><b>Why Vantage?</b></p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Users can utilize Vantage to operationalize their machine learning process. In addition, users have access to AI/ML through:</p>

<ul style = 'font-size:16px;font-family:Arial;color:#00233C'>
    <li>Data Discovery and Statistical Analysis</li>
    <li>Data Preparation and Feature Engineering</li>
    <li>Model Deployment and Evaluation at Operational Scale</li>
</ul>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Traditional approaches require the developer to move data <b>from</b> the sources <b>to</b> the analytics.  Even "integrated" analytic systems like Apache Spark provide parallel processing for analyzing data but don't optimize for loading data - neither locality nor quantity that needs to be moved.</p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Teradata Vantage reverses this model; and allows PUSH processing down to the individual processing nodes where the data resides.  This allows for the unprecedented scale of the analytical processing, reduced costs in data movement/egress charges, and drastically improved performance.</p> 

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>A critical strategy for Vantage and ClearScape Analytics is to embrace the value and innovation in the open-source and partner ML and AI community. A cornerstone of that strategy is to allow users to leverage their ML or AI tools and models of choice to deploy those models directly to the Vantage Platform.  This provides enterprises with the most scalable option for deploying custom machine learning pipelines. Users can leverage the innovation and familiarity of a broad range of tools and techniques, with the ability to prepare and score new data in near-real-time and at any scale; allowing the products of machine learning to become pervasive across all applications, reporting tools, and consumers in an organization.</p>

<hr style="height:2px;border:none;background-color:#00233C;">
<p style = 'font-size:18px;font-family:Arial;color:#00233C'><b>Downloading and installing additional software needed</b>

In [ ]:
# %%capture
# # '%%capture' suppresses the display of installation steps of the following packages
# !pip install sklearn2pmml
# !pip install jdk4py
# !pip install teradataml

<div class="alert alert-block alert-info">
    <p style = 'font-size:16px;font-family:Arial;color:#00233C'><b>Note: </b><i>The above statements may need to be uncommented if you run the notebooks on a platform other than ClearScape Analytics Experience that does not have the libraries installed. If you uncomment those installs, be sure to restart the kernel after executing those lines to bring the installed libraries into memory. The simplest way to restart the Kernel is by typing zero zero: <b> 0 0</b></i></p>
</div>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Here, we import the required libraries, set environment variables and environment paths (if required).</p>

In [ ]:
# Standard library imports
import os
import getpass
import sys
import warnings

# Third-party library imports
from jdk4py import JAVA, JAVA_HOME, JAVA_VERSION

from teradataml import *
from teradatasqlalchemy import FLOAT
from sqlalchemy import func

from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor

from sklearn2pmml.pipeline import PMMLPipeline
from sklearn2pmml import sklearn2pmml

# Suppress warnings
warnings.filterwarnings('ignore')
display.max_rows = 5

# Modify the following to match the specific client environment settings
configure.val_install_location = 'val'
configure.byom_install_location = 'mldb'
os.environ['PATH'] = os.pathsep.join([os.environ['PATH'], str(JAVA_HOME), str(JAVA)[:-5]])

<hr style="height:2px;border:none;background-color:#00233C;">
<b style = 'font-size:20px;font-family:Arial;color:#00233C'>1. Connect to Vantage</b>
<p style = 'font-size:16px;font-family:Arial;color:#00233C'>We will be prompted to provide the password. We will enter the password, press the Enter key, and then use the down arrow to go to the next cell.</p>

In [ ]:
%run -i ../startup.ipynb
eng = create_context(host = 'host.docker.internal', username = 'demo_user', password = password)
print(eng)

In [ ]:
%%capture
execute_sql('''SET query_band='DEMO=Energy_Consumption_Forecasting_Python.ipynb;' UPDATE FOR SESSION;''')

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Begin running steps with Shift + Enter keys. </p>

<p style = 'font-size:20px;font-family:Arial;color:#00233C'><b>Getting Data for This Demo</b></p>
<p style = 'font-size:16px;font-family:Arial;color:#00233C'>We have provided data for this demo on cloud storage. We have the option of either running the demo using foreign tables to access the data without using any storage on our environment or downloading the data to local storage, which may yield somewhat faster execution. However, we need to consider available storage. There are two statements in the following cell, and one is commented out. We may switch which mode we choose by changing the comment string.</p>

In [ ]:
# %run -i ../run_procedure.py "call get_data('DEMO_Energy_cloud');"        # Takes 1 minute
%run -i ../run_procedure.py "call get_data('DEMO_Energy_local');"        # Takes 2 minutes

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Optional step – We should execute the below step only if we want to see the status of databases/tables created and space used.</p>

In [ ]:
%run -i ../run_procedure.py "call space_report();"        # Takes 10 seconds

<hr style="height:2px;border:none;background-color:#00233C;">
<b style = 'font-size:20px;font-family:Arial;color:#00233C'>2. Data Exploration</b>

<table style = 'width:100%;table-layout:fixed;'>
<tr>
    <td style = 'vertical-align:middle' width = '50%'>
        <p style = 'font-size:16px;font-family:Arial;color:#00233C'>We can access large volumes of data by connecting remotely using the teradataml client connection library. Our Python methods are translated to SQL and run remotely on the Vantage system. We only copy the minimal amount of data required to the client, allowing us to interact with data sets of any size and scale.</p>
    </td>
    <td><img src = 'images/connect_and_discover.png' width = '400'></td>
</tr>
</table>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Let's create a "Virtual DataFrame" that points to the data set in Vantage</p>

In [ ]:
tdf = DataFrame(in_schema("DEMO_Energy", "consumption"))
tdf.shape

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>In this demo, we used a dataset that represents electricity consumption in Norway from the 1st of January 2016 to the 31st of August 2019. Each line in our dataset reflects consumption for one hour. Apart from electricity consumption, our dataset also reflects additional data: weather from multiple sources, daylight information, and the labor calendar. We collected all our data from open data sources.</p>
<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Let's investigate the data by looking at a data sample.</p>

In [ ]:
tdf

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>The dataset above shows our hourly consumption of energy. We have multiple columns that are potential factors affecting our energy consumption, such as is_dark, is_holiday, etc.</p> 

In [ ]:
figure = Figure(width=1200, height=600, heading="Energy Demand")

plot = tdf.plot(
    x=tdf.TD_TIMECODE,
    y=tdf.consumption,
    xtick_format='YYYY-MM',
    xlabel='Date',
    ylabel='Energy Units',
    color='carolina blue',
    figure=figure,
    legend='Energy',
    legend_style='upper right',
    grid_linestyle='--',
    grid_linewidth=0.5
)

plot.show()

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Scaling up data visualization presents challenges with processing and interpreting large datasets, causing issues like slow performance. Specialized methods, such as "td_plot," address these challenges by providing efficient solutions for insights without compromising speed.
<br>
<br>
The "td_plot" method in Teradata Vantage simplifies large-scale visualization by allowing users to create visualizations directly within the Vantage environment. It eliminates the need for data movement, enhancing efficiency and addressing challenges associated with handling extensive datasets.</p>

In [ ]:
# Instantiate MinMaxScalar
rs = MinMaxScalar(columns=['cap_air_temperature', 'cap_cloud_area_fraction', 'cap_precipitation_amount'])

# Transform the data using MinMaxScaler
t_output = valib.Transform(data=tdf,
                          rescale=[rs],
                          index_columns='TD_TIMECODE').result

# Create subplots
fig, axis = subplots(3, 1)

# Set figure height and width
fig.height, fig.width = 900, 1200

cols = ['cap_air_temperature', 'cap_cloud_area_fraction', 'cap_precipitation_amount']

# Plot the data
for i in range(len(cols)):
    plot = t_output.plot(x=t_output.TD_TIMECODE,
                         y=t_output[cols[i]],
                         ax=axis[i],
                         xtick_format='YYYY-MM',
                         xlabel='Date',
                         ylabel='Normalized Values',
                         color='carolina blue',
                         figure=fig,
                         legend_style='upper right',
                         grid_linestyle='--',
                         grid_linewidth=0.5)

# Show the plot
plot.show()

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>In our analysis, the graph of cap_air_temperature shows an inverse relationship with our energy consumption. This means that in countries with colder climates like Norway, electricity usage tends to increase as the temperature drops, likely due to increased demand for heating. Conversely, electricity usage tends to decrease when the temperature rises, potentially due to reduced need for heating.</p>

In [ ]:
agg_tdf = tdf.assign(Quarter=func.td_quarter_of_year(tdf.TD_TIMECODE.expression)).filter(items=['consumption', 'Quarter']).groupby('Quarter').mean()

agg_tdf.plot(
    x=agg_tdf.Quarter,
    y=agg_tdf.mean_consumption,
    color='carolina blue',
    kind='bar'
)

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>In our analysis, the above graph shows the distribution of energy consumption across quarters. We observe that the 1st and 4th quarters across years witness high energy consumption due to cold weather, while the 3rd quarter witnesses the least energy consumption across years, indicating the summer season.</p> 

<hr style="height:2px;border:none;background-color:#00233C;">
<b style = 'font-size:20px;font-family:Arial;color:#00233C'>3. Data Preparation</b>

<table style = 'width:100%;table-layout:fixed;'>
<tr>
    <td style = 'vertical-align:top' width = '50%'>
        <p style = 'font-size:16px;font-family:Arial;color:#00233C'>We, as users of the Vantage Analytic Library, benefit from a suite of powerful functions that enable us to perform whole-data-set descriptive analysis, data transformation, hypothesis testing, and algorithmic algorithms at an extreme scale. As with all Vantage capabilities, we run these functions in parallel at the source of the data.</p>
        <ol style = 'font-size:16px;font-family:Arial;color:#00233C'>
            <li>Create Feature Transformation objects</li>
            <br>
            <li>Define the columns to be retained in the analytic data set</li>
            <br>
            <li>Push the transformations to the data in Vantage</li>
            <br>
            <li>Inspect the results</li>
        </ol>
    </td>
    <td><img src = 'images/VAL_transformation.png' width = '400'></td>
</tr>
</table>

In [ ]:
weekday_mapping = {1: 'monday', 2: 'tuesday', 3: 'wednesday', 4: 'thursday', 5: 'friday', 6: 'saturday', 7: 'sunday'}
weekday_t = OneHotEncoder(values=weekday_mapping, columns='weekday')

hour_t = OneHotEncoder(values=[x for x in range(0, 24)], columns='h')

rs = MinMaxScalar(columns=['nasa_temp', 'cap_air_temperature', 'cap_cloud_area_fraction', 'cap_precipitation_amount'])

rt = Retain(columns=['consumption',
                     'is_dark', 'is_light', 'is_from_light_to_dark', 'is_from_dark_to_light',
                     'is_holiday', 'is_pre_holiday'])

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>We will use the transformation objects created in the previous step to prepare our data for modeling. Specifically, we will use weekday_t and hour_t to convert our weekday and hour columns from numeric to one-hot encoded columns. We will use rs to scale our nasa_temp using MinMaxScalar, and rt will be used to retain the specified columns. These transformations will enable us to use our data effectively in a machine learning model..</p>

In [ ]:
t_output = valib.Transform(
    data=tdf,
    one_hot_encode=[weekday_t, hour_t],
    rescale=[rs],
    index_columns='TD_TIMECODE',
    retain=[rt]
)

In [ ]:
t_output.result

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Please scroll to the right and observe that we now have columns named <b>monday-sunday</b> and <b>0_h - 23_h</b>. Also, nasa_temp has been scaled.</p>

In [ ]:
copy_to_sql(t_output.result.iloc[int(t_output.result.shape[0]) - 168:],
            table_name='energy_consumption_variables_rescaled_test',
            if_exists='replace')

copy_to_sql(t_output.result.iloc[:int(t_output.result.shape[0]) - 168],
            table_name='energy_consumption_variables_rescaled_train',
            if_exists='replace')

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>We create training and testing datasets. We use the last 168 hours, i.e., seven days, for testing, and we use the remaining data for training.</p>

<hr style="height:2px;border:none;background-color:#00233C;">
<b style = 'font-size:20px;font-family:Arial;color:#00233C'>4. Model Training</b>

<table style = 'width:100%;table-layout:fixed;'>
<tr>
    <td style = 'vertical-align:top' width = '50%'>
        <p style = 'font-size:16px;font-family:Arial;color:#00233C'>With Vantage Bring Your Own Model, we can take advantage of a rich ecosystem of Machine Learning, Data Preparation, and Advanced analytical libraries available in the open-source and commercial space. This demonstration illustrates how we can utilize simple client-side training pipelines.</p>
        <ol style = 'font-size:16px;font-family:Arial;color:#00233C'>
            <li>Copy the training data to the client</li>
            <br>
            <li>Prepare data and train the model</li>
            <br>
            <li>Load the model into Vantage</li>
        </ol>
    </td>
    <td><img src = 'images/BYOM_model_training.png' width = '400'></td>
</tr>
</table>

<div class="alert alert-block alert-info">
    <p style = 'font-size:16px;font-family:Arial;color:#00233C'><b>Note: </b><i>Please remember that in this section, we move data from Vantage to client side <b>only to simulate model being trained outside Vantage</b>. The trained model would be used to demonstrate BYOM (Bring Your Own Model) functionality of Vantage.</i></p>
</div>

In [ ]:
pandas_df = DataFrame('energy_consumption_variables_rescaled_train') \
    .sort('TD_TIMECODE') \
    .to_pandas(all_rows=True)

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>We calculate the average consumption for the last day of the training period. We will use this number for the normalization of the target variable.</p>

In [ ]:
normalize_value = int(pandas_df.tail(24).mean()['consumption'])
normalize_value

In [ ]:
train_x = pandas_df.drop(['TD_TIMECODE', 'consumption'], axis=1).astype(float)
train_y = pandas_df['consumption'] - normalize_value

feature_names = list(train_x.columns)

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>We drop the TD_TIMECODE and consumption columns from our training dataset because TD_TIMECODE is a timestamp, and consumption is the target variable we aim to predict. Therefore, we can't use it as a feature in our model. We normalize the target variable by subtracting the normalized_value to scale the consumption values and bring them into a similar range as the other features used for prediction. This approach helps us in learning the correct relationship between features and the target variable, making our training process more efficient.</p>

In [ ]:
pipeline_lr = PMMLPipeline(steps=[('lr', LinearRegression())])
pipeline_rf = PMMLPipeline(steps=[('random_forest', RandomForestRegressor(n_estimators=100, max_depth=10, random_state=42))])

pipeline_lr.fit(train_x, train_y)
pipeline_rf.fit(train_x, train_y)

sklearn2pmml(pipeline_lr, "energy_consumption_LR.pmml", with_repr=True)
sklearn2pmml(pipeline_rf, "energy_consumption_RF.pmml", with_repr=True)

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>In the above step, we create two PMML Pipelines, one with a Linear Regression object and another with a Random Forest object. We use these Pipelines to train our models using the "fit" method with the preprocessed training dataset. After training, we save the trained pipelines as PMML files locally for future use.</p>

In [ ]:
# Load the PMML file into Vantage

model_ids = ['lr', 'rf']
model_files = ['energy_consumption_LR.pmml', 'energy_consumption_RF.pmml']
table_name = 'energy_models'

for model_id, model_file in zip(model_ids, model_files):
    try:
        save_byom(model_id=model_id, model_file=model_file, table_name=table_name)
    except Exception as e:
        # If our model exists, delete and rewrite
        if str(e.args).find('TDML_2200') >= 1:
            delete_byom(model_id=model_id, table_name=table_name)
            save_byom(model_id=model_id, model_file=model_file, table_name=table_name)
        else:
            raise ValueError(f"Unable to save the model '{model_id}' in '{table_name}' due to the following error: {e}")

# Show the model table
list_byom(table_name)

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>In the above steps, we save our trained PMML models in a table named <b>energy_models</b>. If a model with the same model_id already exists, we delete it first, and then we save the latest trained model again using the <b>save_byom</b> method. This ensures that we always store the most recent version of the model in the table.</p>

<hr style="height:2px;border:none;background-color:#00233C;">
<b style = 'font-size:20px;font-family:Arial;color:#00233C'>5. Model Scoring and Evaluation</b>

<table style = 'width:100%;table-layout:fixed;'>
<tr>
    <td style = 'vertical-align:top' width = '50%'>
        <p style = 'font-size:16px;font-family:Arial;color:#00233C'>The final step in our process is to test the trained model. We will use the PMMLPredict function to take our stored pipeline object (including any data preparation and mapping tasks) and execute it against the data on the Vantage Nodes. Note that we can keep many models in our model table, with versioning, last scored timestamp, or any other management data to allow for the operational management of our process.</p>
        <ol style = 'font-size:16px;font-family:Arial;color:#00233C'>
            <li>Create a pointer to the model in Vantage</li>
            <br>
            <li>Execute the Scoring function using the model against the testing data</li>
            <br>
            <li>Copy the results of the test to the client - only needs to be a subset of rows if desired</li>
            <br>
            <li>Visualize the results</li>
        </ol>
    </td>
    <td><img src = 'images/Score_and_Evaluate.png' width = '400'></td>
</tr>
</table>

In [ ]:
# Obtain a pointer to the model
table_name = 'energy_models'
model_id = 'lr'
model_lr = retrieve_byom(model_id, table_name=table_name)
df_test = DataFrame('energy_consumption_variables_rescaled_test')

result_lr = PMMLPredict(
    modeldata=model_lr,
    newdata=df_test,
    accumulate=['TD_TIMECODE', 'consumption'],
).result

result_lr = result_lr.assign(prediction=result_lr.prediction.cast(type_=FLOAT()))
result_lr = result_lr.assign(prediction=result_lr.prediction + normalize_value)

In [ ]:
result_lr

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>In the above step, we use the PMMLPredict method from the teradataml library to score our model in the database. The PMMLPredict function in Teradata allows us to score the PMML model directly on the data in the Vantage system, without having to move the data or the model outside the system. This can help to improve our efficiency and security in the scoring process.</p>

In [ ]:
# Obtain a pointer to the model
table_name = 'energy_models'
model_id = 'rf'
model_rf = retrieve_byom(model_id, table_name=table_name)
df_test = DataFrame('energy_consumption_variables_rescaled_test')

result_rf = PMMLPredict(
    modeldata=model_rf,
    newdata=df_test,
    accumulate=['TD_TIMECODE', 'consumption'],
).result

result_rf = result_rf.assign(prediction=result_rf.prediction.cast(type_=FLOAT()))
result_rf = result_rf.assign(prediction=result_rf.prediction + normalize_value)

In [ ]:
result_rf

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>In the above step, we use the PMMLPredict method from the teradataml library to score our model in the database. The PMMLPredict function in Teradata allows us to score the PMML model directly on the data in our Vantage system, without having to move the data or the model outside the system. This can help to improve the efficiency and security of our scoring process.</p>

<hr style="height:2px;border:none;background-color:#00233C;">
<b style = 'font-size:20px;font-family:Arial;color:#00233C'>6. Visualize the results</b>

In [ ]:
# Calculate RMS errors
rms_lr = RegressionEvaluator(data=result_lr, 
                             observation_column="consumption",
                             prediction_column="prediction",
                             metrics=['RMSE']).result.get_values()[0][0]

rms_rf = RegressionEvaluator(data=result_rf, 
                             observation_column="consumption",
                             prediction_column="prediction",
                             metrics=['RMSE']).result.get_values()[0][0]

In [ ]:
figure = Figure(width=1400, height=600,  heading="Energy Consumption Prediction")

df_test.plot(
                x=df_test.TD_TIMECODE,
                y=[df_test.consumption, result_lr.prediction, result_rf.prediction],
                figure=figure,
                xtick_format='YYYY-MM-DD',
                xlabel='TD_TIMECODE',
                ylabel='Energy Consumption',
                legend=['Actual Consumption', f'Linear Regression (RMS = {rms_lr:.2f})', f'Random Forest (RMS = {rms_rf:.2f})'],
                legend_style='upper right',
                grid_linestyle='--',
                grid_linewidth=0.5,
                linestyle=['-', '--', '--']
)

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>The above graph displays the Root Mean Squared (RMS) error values for both our Linear Regression and Random Forest models. The lower the RMS error value, the better our models' performance. As we can see, Random Forest outperforms Linear Regression in predicting energy demand, as it has a lower RMS error value. Therefore, Random Forest is more suitable for proactively predicting energy demand in our use case.</p>

<p style = 'font-size:18px;font-family:Arial;color:#00233C'><b>Conclusion</b></p>
<p style = 'font-size:16px;font-family:Arial;color:#00233C'>In this demonstration, we have illustrated a simplified - but complete - overview of how we can improve a typical machine learning workflow using Vantage in conjunction with open-source tools and techniques. This combination allows us to leverage open-source innovation with Vantage's operational scale, power, and stability.</p>

<hr style="height:2px;border:none;background-color:#00233C;">
<b style = 'font-size:20px;font-family:Arial;color:#00233C'>7. Cleanup</b>

<p style = 'font-size:18px;font-family:Arial;color:#00233C'><b>Work Tables</b></p>
<p style = 'font-size:16px;font-family:Arial;color:#00233C'>We need to clean up our work tables to prevent errors next time.</p>

In [ ]:
tables = ['energy_models', 'energy_consumption_variables_rescaled_train', 'energy_consumption_variables_rescaled_test']

# Loop through the list of tables and execute the drop table command for each table
for table in tables:
    try:
        db_drop_table(table_name = table)
    except:
        pass

<p style = 'font-size:18px;font-family:Arial;color:#00233C'> <b>Databases and Tables</b></p>
<p style = 'font-size:16px;font-family:Arial;color:#00233C'>We will use the following code to clean up tables and databases created for this demonstration.</p>

In [ ]:
%run -i ../run_procedure.py "call remove_data('DEMO_Energy');"        # Takes 5 seconds

In [ ]:
remove_context()

<hr style="height:2px;border:none;background-color:#00233C;">

<b style = 'font-size:20px;font-family:Arial;color:#00233C'>Required Materials</b>
<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Let’s look at the elements we have available for reference for this notebook:</p>


<p style = 'font-size:18px;font-family:Arial;color:#00233C'><b>Filters:</b></p>
<ul style = 'font-size:16px;font-family:Arial;color:#00233C'>
    <li><b>Industry:</b> Energy and Natural Resource</li>
    <li><b>Functionality:</b> Open and Connected Analytics</li>
    <li><b>Use Case:</b> Bring Your Own Model</li>
</ul>

<p style = 'font-size:18px;font-family:Arial;color:#00233C'><b>Related Resources:</b></p>

<ul style = 'font-size:16px;font-family:Arial;color:#00233C'>
    <li><a href='https://www.teradata.com/Blogs/Digital-utilities-driving-value-from-analytics'>Digital utilities driving value from advanced analytics at scale</a></li>
    <li><a href='https://www.teradata.com/Blogs/Hyper-scale-time-series-forecasting-done-right'>Hyper-scale time series forecasting done right</a></li>
    <li><a href='https://www.teradata.com/Industries/Energy-and-Natural-Resources'>Gain Analytic Insights for the Energy Industry</a></li>
</ul>

<footer style="padding-bottom:35px; background:#f9f9f9; border-bottom:3px solid #00233C">
    <div style="float:left;margin-top:14px">ClearScape Analytics™</div>
    <div style="float:right;">
        <div style="float:left; margin-top:14px">
            Copyright © Teradata Corporation - 2023. All Rights Reserved
        </div>
    </div>
</footer>